<a href="https://colab.research.google.com/github/DavidDimasPatty/API_GO_LEARN/blob/master/reelsADS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download Model**


In [ ]:
!pip install diffusers transformers accelerate torch torchvision imageio[ffmpeg]

**Import**

In [ ]:
import torch
from diffusers.pipelines.pipeline_utils import DiffusionPipeline
import imageio
import numpy as np
import os
import cv2
import datetime

**Connection**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_id = "cerspense/zeroscope_v2_576w"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

pipe.enable_model_cpu_offload()
pipe.enable_attention_slicing()

prompt = "a cinematic drone shot of a futuristic city at sunset"

video_frames = pipe(
    prompt,
    num_frames=32,
    height=256,
    width=448,
    num_inference_steps=25
).frames
processed_frames = []

for frame in video_frames:

    if isinstance(frame, torch.Tensor):
        frame = frame.detach().cpu().numpy()

    frame = np.array(frame)

    # Kalau 4D (kadang ada batch)
    if frame.ndim == 4:
        frame = frame[0]

    # Kalau format (C,H,W)
    if frame.shape[0] == 3:
        frame = np.transpose(frame, (1,2,0))

    # Kalau grayscale (H,W)
    if frame.ndim == 2:
        frame = np.stack([frame]*3, axis=-1)

    # Convert ke uint8
    if frame.max() <= 1.0:
        frame = (frame * 255).clip(0,255)

    frame = frame.astype(np.uint8)

    processed_frames.append(frame)

# --- SAVE DENGAN OPENCV ---

output_path = f"/content/drive/MyDrive/ADS_Video/generated_reel_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.mp4"

h, w, _ = processed_frames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 8, (w, h))

for frame in processed_frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

out.release()

print("Video saved:", output_path)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Loading weights:   0%|          | 0/372 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after Diffusers version 0.33.1. 


  0%|          | 0/25 [00:00<?, ?it/s]

AttributeError: module 'datetime' has no attribute 'now'